# Information-Scraper

The Information-Scraper was created to scrape all the sub-links of advertisements of used cars offered via platform https://www.autoscout24.com/ and extract the various details about random selected vehicles.For the currect porposes the UsedCars_Downloade was set to download the vehicles of the most famous brands offered in Austria. (Note that scraper alover a user to change this setting). 

In [28]:
from random import sample 
import sqlite3 as sql
import pandas as pd


from bs4 import BeautifulSoup
import math
import tqdm
import json
import pandas as pd
import datetime
import time

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from IPython.core.debugger import Tracer
import time
from random import sample 

In [29]:
class UsedCars_Downloader:  
    '''
    UsedCars_Downloader was created to scrape all the sub-links of advertisements of used cars offered via platform 
    https://www.autoscout24.com/ and extract the various details about 
    random selected vehicles. 
    For the currect porposes the Downloader was set to download the vehicles of the most famous brands offered in Austria.
    (Note that scraper alover a user to change this setting)
    '''
    def getSoup(self, link):
        r = requests.get(link)
        r.encoding = 'UTF-8'
        return BeautifulSoup(r.text,'lxml')

    def getAllValues(self, link):
        '''
        getAllValues scrapes the information regarding the features get by getAllFeatures 
        (for instance the value of technical features, colour etc.)  
        '''
        soup = self.getSoup(link)
        qs= soup.findAll('dd')
        return [q.text[1:-1] for q in qs]

    def getAllFeatures(self, link):
        '''
        getAllFeature crapes the available variables names available for the selected cars. getAllValues than scrape the exact values
        '''
        soup = self.getSoup(link)
        zs = soup.findAll('dt')
        return [z.text for z in zs]



    def getTitle(self, link):
        '''
        getTitle scrapes the name of the selected advertisement that in many cases refere to the name of a vehicle and
        the most important categories that should attract a potentional buyer
        '''
        soup = self.getSoup(link)
        return soup.find('h2').text


    def getPrice(self, link):
        '''
        getPrice scrapes the ask price of a vehicle
        '''
        soup = self.getSoup(link)
        return soup.find('div', {"class":"cldt-price"}).find('h2').text[1:-1]
    
    def getPriceLabel(self, link):
        '''
        getPriceLabel scrapes the pricelabel of a vehicle
        '''
        soup = self.getSoup(link)
        return soup.find('data-pricelabel')

    def getDealerType(self, link):
        '''
        getDealerType scrapes if private or dealer
        '''
        soup = self.getSoup(link)
        extract = soup.find('div', {"class":"cldt-stage-vendor-data"})
        xs = extract.findAll('span')
        return [x.text for x in xs]

    def getVat(self, link):
        '''
        getVat scrapes if vat deductable
        '''
        soup = self.getSoup(link)
        result = soup.find('p', {"sc-font-s cldt-price-info sc-padding-bottom-xl"}).find('span')
        

        if result is not None:
            title = result.get_text()
        else : title = result 
        
        return title

    def getPrimaryspecs(self, link):
        '''
        getPrimaryspecs scrapes primary specification of a car such as mileage, age etc. 
        '''
        soup = self.getSoup(link)
        a = soup.find('div', {"class":"cldt-stage-basic-data"}).findAll('div')[0].find('span').text
        b = soup.find('div', {"class":"cldt-stage-basic-data"}).findAll('div')[1].find('span').text
        c = soup.find('div', {"class":"cldt-stage-basic-data"}).findAll('div')[2].find('span').text
        d = soup.find('div', {"class":"cldt-stage-basic-data"}).findAll('div')[3].find('span').text


        return a,b,c,d 
    
    def getEqui(self, link):
        '''
        getPrimaryspecs scrapes the Equipment section and finds all in span
        '''
        soup = self.getSoup(link)
        extract = soup.find('div', {"class":"cldt-equipment-block sc-grid-col-3 sc-grid-col-m-4 sc-grid-col-s-12 sc-pull-left"})
        xs = extract.findAll('span')
        return [x.text for x in xs]

        
    def getPrimaryspecs2(self, link):
        '''
        getPrimaryspecs scrapes the rest of primary specification of a vehicle
        '''
        soup = self.getSoup(link)
        extract = soup.find('div', {"class":"cldt-stage-basic-data"}).findAll('div')[3]
        xs = extract.findAll('span')
        return [x.text for x in xs]


    def RandomSample(self, df, coef):
        '''
        RandomSample creates a random selection within given data and selected amount of outputs
        '''
        x=sample(df, coef)
        return x
    
    def checker2(self, df):  
        l=[]
        for link in df:
            checker=requests.get(link)
            if str(checker) == '<Response [410]>':
                check = 'False'
            if str(checker) == '<Response [200]>':
                check = 'True'
            else : check = 'False'
            d = {
                'link':link,
                'Response':check,
        
            
                }
            l.append(d)
        D=pd.DataFrame(l)
        exists =  D['Response']=='True'
        DF=D[exists]
        DF= DF['link'].values.tolist()
        return DF
            
        
    def scraper(self, links, nmbroflinks, grouplen):
        start= time.time()
        tracker=0

        database = 'info27_01_2021.db'
        connection = sql.connect(database)
        l = []
        for link in links[0:nmbroflinks]:
            Features=self.getAllFeatures(link)
            Values=self.getAllValues(link)
            Title=self.getTitle(link)
            Price=self.getPrice(link) 
            PriceLabel=self.getPriceLabel(link)
            DealerType=self.getDealerType(link)
            Primaryspecs=self.getPrimaryspecs(link)
            Primaryspecs2=self.getPrimaryspecs2(link) 
            Equi=self.getEqui(link) 
            Vat=self.getVat(link) 
            d = {
            'link':link,
            'Title':Title,
            'Price':Price,
            'PriceLabel':PriceLabel,
            'Vat':Vat,    
            'Dealertype':DealerType[2],
            'km':Primaryspecs[0],
            'Zustand':Primaryspecs[1],
            'Registration date':Primaryspecs[2],
            'kW':Primaryspecs[3],
            'Equi':Equi,
            'kW':Primaryspecs2[0],
            'hp': Primaryspecs2[1],
            'transmission': Primaryspecs2[2],
            'Ffuel': Primaryspecs2[2],
            Features[0]:Values[0],
            Features[1]:Values[1],
            Features[2]:Values[2],
            Features[3]:Values[3],
            Features[4]:Values[4],
            Features[5]:Values[5]
            }
            try:
                d[Features[6]] = Values[6]
            except:
                pass
            try:
                d[Features[7]] = Values[7]
            except:
                pass
            try:
                d[Features[8]] = Values[8]
            except:
                pass
            try:
                d[Features[9]] = Values[9]
            except:
                pass
            try:
                d[Features[10]] = Values[10]
            except:
                pass
            try:
                d[Features[11]] = Values[11]
            except:
                pass
            try:
                d[Features[12]] = Values[12]
            except:
                pass
            try:
                d[Features[13]] = Values[13]
            except:
                pass
            try:
                d[Features[14]] = Values[14]
            except:
                pass
            try:
                d[Features[15]] = Values[15]
            except:
                pass
            try:
                d[Features[16]] = Values[16]
            except:
                pass
            try:
                d[Features[17]] = Values[17]
            except:
                pass
            try:
                d[Features[18]] = Values[18]
            except:
                pass
            try:
                d[Features[19]] = Values[19]
            except:
                pass
            try:
                d[Features[20]] = Values[20]
            except:
                pass
            try:
                d[Features[21]] = Values[21]
            except:
                pass
            try:
                d[Features[22]] = Values[22]
            except:
                pass
            
            l.append(d)
            
            tracker = tracker +1

            if tracker % grouplen == 0:
                
                info1 =pd.DataFrame(l)
                info1 = info1.applymap(str)
                try:
                    info1.to_sql('toupload_rh3', connection, if_exists='append')
                except:
                    data=pd.read_sql_query('''SELECT * from toupload_rh3''', connection)
                    df2=pd.concat([data, info1])
                    df2.to_sql('toupload_rh3', connection, if_exists='replace')
                l = []
            time.sleep(1)



        print(tracker)           
        end=time.time()            
        print(end - start)
        print(info1)


# Data inputs

In [30]:
database = 'DEdatabase2.db'
connection = sql.connect(database)
'''
#cursor = connection.cursor()
#renameTable = "ALTER TABLE DEsportlinksupto100k RENAME COLUMN '0' TO 'links'"

cursor.execute(renameTable)

#renameTable = "ALTER TABLE DEsportupto120k RENAME COLUMN '0' TO 'links'"

cursor.execute(renameTable)
'''
df = pd.read_sql_query('''SELECT * from DEsportlinksupto100k''', connection)
print(df.columns.tolist())
df= df['0'].values.tolist()


df

['index', '0']


['https://www.autoscout24.de/angebote/audi-rs6-bose-21-carbon-dynamik-hud-garantie-sehr-gepflegt-benzin-schwarz-b165adb9-4d07-706a-e053-0100007f6e42?',
 'https://www.autoscout24.de/angebote/audi-a6-2-4-benzin-blau-6b640bce-7979-4fcb-803e-d1b409edfb6a?',
 'https://www.autoscout24.de/angebote/audi-tt-rs-coupe-benzin-weiss-a80521ff-0c7c-4671-9ced-67cd1ac7f9fe?',
 'https://www.autoscout24.de/angebote/audi-a6-avant-sport-45-tdi-quattro-180-245-kw-ps-s-diesel-grau-a3286017-2669-441b-9b28-ca8220524883?',
 'https://www.autoscout24.de/angebote/bmw-m5-competition-benzin-blau-3817332c-a9e3-4cb2-99fb-c1ad49c167c4?',
 'https://www.autoscout24.de/angebote/bmw-m850-i-xdrive-cabrio-benzin-grau-e2e786af-51dd-4a65-abe0-f07f805193e2?',
 'https://www.autoscout24.de/angebote/bmw-x5-xdrive30d-diesel-weiss-ecf86232-06f5-4314-b144-035e0fb2d591?',
 'https://www.autoscout24.de/angebote/bmw-x4-xdrive20d-x-line-diesel-grau-78082386-2c69-4b13-9cbc-3ed3bad85061?',
 'https://www.autoscout24.de/angebote/bmw-x3-xdrive

## Random Sampling and Cleaning of the Nonexisting Sub-links 

Random Sampling: According to the needs of the user of the code, the Random Sample that will be scraped can be changed by the coef value.

Nonexisting Sub-links: The Carlinks-Scraper takes long time and between the start of the scraping of sublinks and its end could some of those links be deleted. For this reason it is necessary to run the following information-scraping functions only on the existing sub-links. Checker2 can do this procedure. 

In [31]:
usedcars=UsedCars_Downloader()

In [32]:
coef=20

In [33]:
DF=usedcars.checker2(usedcars.RandomSample(df, coef))
len(DF)
DF

['https://www.autoscout24.de/angebote/bmw-x6-xdrive30d-m-sport-22-lm-standheizung-ahk-gsd-diesel-weiss-3515787e-d1b3-46df-b9eb-c8f12d885204?',
 'https://www.autoscout24.de/angebote/audi-e-tron-sportback-s-line-55-quattro-300-kw-matrix-led-pan-elektro-blau-5ffe9561-2b97-479a-8766-372d251029d9?',
 'https://www.autoscout24.de/angebote/audi-s6-diesel-grau-70f965ad-5a3c-4e66-9393-b4c98d5862ba?',
 'https://www.autoscout24.de/angebote/audi-q8-50-tdi-quattro-tiptronic-ahk-matrix-navi-shz-diesel-weiss-e68fbbe3-68c6-492f-816b-259a2ae9ced7?',
 'https://www.autoscout24.de/angebote/bmw-x5-m-50-i-innovationsp-sport-aut-komfortsitze-benzin-weiss-bcacc83e-e038-4ffa-85be-00fbe4d4e857?',
 'https://www.autoscout24.de/angebote/bmw-m850-i-xdrive-cabrio-innovation-paket-benzin-weiss-c3463daa-4e16-4a1f-b78a-36af93e15219?',
 'https://www.autoscout24.de/angebote/audi-e-tron-sportback-s-line-55-quattro-klima-led-navi-leder-elektro-weiss-b956c930-e4ea-4208-912a-a4723d22659d?',
 'https://www.autoscout24.de/angebo

## Scraping

According to the needs of the user the nmbroflinks and grouplen can be changed. This part will ensure to optimize the equilibrium between the scraping speed and the semi-intervals of storing the data. 

Nmboflinks defines the final number of links that have to be scrapped.

In [34]:
nmbroflinks=len(DF)

Grouplen defines the number of links that are scraped within one loop.

In [35]:
len(DF)

19

In [36]:
grouplen=len(DF)

In [37]:
usedcars.scraper(DF, nmbroflinks, grouplen)

AttributeError: 'NoneType' object has no attribute 'findAll'